In [2]:
import duckdb

In [3]:
cn = duckdb.connect()

In [4]:
df = cn.execute("select 1 as x").arrow()

x: int32


In [ ]:
%pip install sqlalchemy pyarrow pyiceberg

### Notes as of 9/3/2024
this is all still a hot mess.

looks like this is a decent tutorial though [link](https://medium.com/@tglawless/getting-started-with-pyiceberg-a2fc1caffdab)

i think the kicker is i should make a yaml file with my iceberg warehouse config like the tutorial did above.

then when setting up the catalog with ("local") as the catalog, it will grab the yaml config

In [1]:
from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import StringType, IntegerType
from pyiceberg.schema import Field, NestedField

In [2]:
schema = Schema(
        NestedField(field_id=1, name="name", field_type=StringType(), required=True),
        NestedField(field_id=2, name="age", field_type=IntegerType(), required=True)
)

In [7]:
warehouse_path = "./icehouse"
catalog = load_catalog(name="local", uri=f"sqlite:///{warehouse_path}/pyiceberg_catalog.db", warehouse=f"file://{warehouse_path}")

In [8]:
catalog.create_namespace_if_not_exists("junk")

In [9]:
ice_table = catalog.create_table_if_not_exists(identifier="junk.dummy_table",schema=schema, location=warehouse_path)

### Notes

PyIceberg still seems to be very much in its infancy
- creating an iceberg table requires sqlalchemy
- the tables generated do not play nice with the duckdb extension
- when trying to query an iceberg table written by pyiceberg in duckdb, duckdb is looking for several addtional files that pyiceberg does not generate such as a "version-hint.txt" file

In [6]:
from pyiceberg.catalog.sql import SqlCatalog

warehouse_path = "./icehouse"
catalog = SqlCatalog(
    "default",
    **{
        "uri": f"sqlite:///{warehouse_path}/pyiceberg_catalog.db",
        "warehouse": f"file://{warehouse_path}",
    },
)

In [8]:
import requests
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
output_path = "./yellow_tripdata_2023-01.parquet"

response = requests.get(url)
with open(output_path, 'wb') as file:
    file.write(response.content)

In [10]:
import pyarrow.parquet as pq

df = pq.read_table("/tmp/yellow_tripdata_2023-01.parquet")

In [11]:
print(df.schema)

VendorID: int64
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: double
trip_distance: double
RatecodeID: double
store_and_fwd_flag: string
PULocationID: int64
DOLocationID: int64
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2492


In [7]:
catalog.create_namespace("default")

table = catalog.create_table(
    "default.taxi_dataset",
    schema=df.schema,
)

NamespaceAlreadyExistsError: Namespace default already exists

In [10]:
table = catalog.create_table(
    "default.ducks",
    schema=df.schema,
)

In [11]:
table.append(df)
len(table.scan().to_arrow())

1

In [19]:
import pyarrow.compute as pc

df = df.append_column("tip_per_mile", pc.divide(df["tip_amount"], df["trip_distance"]))

In [20]:
with table.update_schema() as update_schema:
    update_schema.union_by_name(df.schema)

In [ ]:
table.overwrite(df)
print(table.scan().to_arrow())

In [22]:
df = table.scan(row_filter="tip_per_mile > 0").to_arrow()
len(df)

2371784

In [24]:
import duckdb

In [25]:
cn = duckdb.connect()

In [26]:
cn.execute("""
    INSTALL iceberg;
    LOAD iceberg;
""")

In [33]:
sql = """
SELECT COUNT(*)
FROM iceberg_metadata('/Users/matthewmartin/dream_machine/iceberg/pyicy/icehouse/default.db/taxi_dataset', allow_moved_paths= true)
"""

#/Users/matthewmartin/dream_machine/iceberg/pyicy/icehouse/default.db/taxi_dataset

cn.sql(sql).show()

IOException: IO Error: Cannot open file "/Users/matthewmartin/dream_machine/iceberg/pyicy/icehouse/default.db/taxi_dataset/metadata/v2.metadata.json": No such file or directory